In [1]:
import sys
import os
import numpy as np
import dill
import gzip
import logging


sys.path.append(os.path.abspath('../../surmise/emulationmethods'))
from AKSGP import Emulator

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        # logging.FileHandler('emulator_train.log', mode='w'),  # Log to file
        logging.StreamHandler()  # Log to console
    ]
)
logger = logging.getLogger(__name__)

# Try mock data

In [2]:
from sklearn.datasets import make_friedman1, make_friedman2
X, y = make_friedman2(n_samples=50, noise=0.5, random_state=0)
emu = Emulator(X=X, Y_mean=y, Y_std=None)
emu.fit(kernel='AKS', nrestarts=5)

Xtest, ytest = make_friedman2(n_samples=10, noise=0.5, random_state=0)
means, std_devs = emu.predict(Xtest)
print("% error:\n", (1.0 - means/ytest.reshape(-1, 1))*100)

2024-08-24 13:18:07,389 - AKSGP - INFO - Automatic kernel selection opted. Best kernel for each output dimension will be selected from the list of kernels:
   ['Matern12', 'Matern32', 'Matern52', 'RBF']

2024-08-24 13:18:07,390 - AKSGP - INFO - Shape of training arrays: (45, 4), (45, 1), (45, 1)
2024-08-24 13:18:07,390 - AKSGP - INFO - Shape of validation arrays: (5, 4), (5, 1), (5, 1)
2024-08-24 13:18:07,390 - AKSGP - INFO - Training GPs with all available kernels...
2024-08-24 13:18:07,391 - AKSGP - INFO -   Standardizing input space...
2024-08-24 13:18:07,391 - AKSGP - INFO -   Standardizing data...
/Users/jaiswal/miniconda3/envs/jssims_new/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/jaiswal/miniconda3/envs/jssims_new/lib/python3.1

% error:
 [[-0.12841442]
 [-0.08725827]
 [ 0.06623413]
 [ 0.22912221]
 [ 0.07472758]
 [-0.05452346]
 [-0.39750841]
 [-0.0890982 ]
 [ 0.1812712 ]
 [-0.10571229]]


# Load heavy-ion data

In [3]:
# Load training data
train_dir = 'simulation_data/Grad_Pb-Pb-2760GeV/train'
X = np.loadtxt(os.path.join(train_dir, 'X.txt'))
Ymean = np.loadtxt(os.path.join(train_dir, 'Ymean.txt'))
Ystd = np.loadtxt(os.path.join(train_dir, 'Ystd.txt'))

print(f"Arrays loaded from directory '{train_dir}'.")
print(f"Shapes of loaded arrays: {X.shape}, {Ymean.shape}, {Ystd.shape}")


# Load testing data
test_dir = 'simulation_data/Grad_Pb-Pb-2760GeV/test'

Xval = np.loadtxt(os.path.join(test_dir, 'X.txt'))
Ymeanval = np.loadtxt(os.path.join(test_dir, 'Ymean.txt'))
Ystdval = np.loadtxt(os.path.join(test_dir, 'Ystd.txt'))

print(f"Arrays loaded from directory '{test_dir}'.")
print(f"Shapes of loaded arrays: {Xval.shape}, {Ymeanval.shape}, {Ystdval.shape}")


Arrays loaded from directory 'simulation_data/Grad_Pb-Pb-2760GeV/train'.
Shapes of loaded arrays: (485, 17), (485, 110), (485, 110)
Arrays loaded from directory 'simulation_data/Grad_Pb-Pb-2760GeV/test'.
Shapes of loaded arrays: (93, 17), (93, 110), (93, 110)


# Train emulators

In [5]:
inpsize = 485; numobs= (0,4,7,34);

emu = Emulator(X=X[:inpsize], Y_mean=Ymean[:inpsize, numobs], Y_std=Ystd[:inpsize, numobs])
emu.fit(kernel='AKS', nrestarts=5, seed=13)


2024-08-24 13:18:40,770 - AKSGP - INFO - Automatic kernel selection opted. Best kernel for each output dimension will be selected from the list of kernels:
   ['Matern12', 'Matern32', 'Matern52', 'RBF']

2024-08-24 13:18:40,771 - AKSGP - INFO - Shape of training arrays: (436, 17), (436, 4), (436, 4)
2024-08-24 13:18:40,772 - AKSGP - INFO - Shape of validation arrays: (49, 17), (49, 4), (49, 4)
2024-08-24 13:18:40,772 - AKSGP - INFO - Training GPs with all available kernels...
2024-08-24 13:18:40,772 - AKSGP - INFO -   Standardizing input space...
2024-08-24 13:18:40,774 - AKSGP - INFO -   Standardizing data...
2024-08-24 13:18:50,111 - AKSGP - INFO -   Trained GPs with Matern12 kernels.
2024-08-24 13:18:56,284 - AKSGP - INFO -   Trained GPs with Matern32 kernels.
2024-08-24 13:19:03,411 - AKSGP - INFO -   Trained GPs with Matern52 kernels.
2024-08-24 13:19:09,917 - AKSGP - INFO -   Trained GPs with RBF kernels.
2024-08-24 13:19:09,918 - AKSGP - INFO - Finding best kernels for each outp

In [6]:
test = 10

print("Predictions on training set ----------------->")
# Predict with standard deviation
means, std_devs = emu.predict(X[:test])

print("  Original means:\n", Ymean[:test, numobs],"\n")
print("  Predicted means:\n", means,"\n")

print("  Original standard deviations:\n", Ystd[:test, numobs],"\n")
print("  Predicted standard deviations:\n", std_devs,"\n")


print("Predictions on test set ----------------->")
# Predict with standard deviation
means, std_devs = emu.predict(Xval[:test])

print("  Original means:\n", Ymeanval[:test, numobs],"\n")
print("  Predicted means:\n", means,"\n")

print("  Original standard deviations:\n", Ystdval[:test, numobs],"\n")
print("  Predicted standard deviations:\n", std_devs,"\n")


Predictions on training set ----------------->
  Original means:
 [[1608.155095  402.4201     64.998956  370.298827]
 [1376.107143  398.155003   75.674266  371.891188]
 [1715.649143  495.888734   93.457043  462.45653 ]
 [1474.812429  439.662816   79.33832   407.198466]
 [1625.855467  504.200066  100.375664  467.423269]
 [1504.085667  379.364243   58.23383   346.128945]
 [1480.250667  443.19724    87.351097  413.624828]
 [1437.252714  427.646196   72.45078   393.168612]
 [1482.820619  364.887598   56.401625  338.102267]
 [1657.787762  484.448914   82.403924  450.87281 ]] 

  Predicted means:
 [[1614.47095375  407.37921578   65.53142559  374.69780572]
 [1371.07007182  397.33566339   74.88825894  371.17218365]
 [1700.72351665  492.83212606   92.42994513  459.42087948]
 [1476.81608483  438.74938252   79.06886704  406.33153474]
 [1627.08074931  504.74109086  100.41187849  467.89392159]
 [1501.80859662  378.84572174   58.28740133  345.68540068]
 [1479.86403955  443.79575979   87.86845277  41